<a href="https://colab.research.google.com/github/ChiroDeniro/ai-ml-projecten/blob/main/learn_Geo_Recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages (run this cell first in Colab)
!pip install gradio pandas numpy plotly pycountry Levenshtein -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 18.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import gradio as gr
from datetime import datetime, timedelta
import json
import random
from collections import defaultdict
import Levenshtein
import pycountry

class SpacedRepetitionLearner:
    """
    Implements SM-2 (SuperMemo 2) algorithm for optimal learning
    """

    def __init__(self):
        self.items = {}  # item_id -> {easiness, interval, repetitions, next_review, history}

    def add_item(self, item_id):
        """Initialize a new learning item"""
        if item_id not in self.items:
            self.items[item_id] = {
                'easiness': 2.5,  # Initial easiness factor
                'interval': 0,    # Days until next review
                'repetitions': 0, # Number of successful repetitions
                'next_review': datetime.now(),
                'history': [],
                'mastery_level': 0  # 0-100 scale
            }

    def calculate_quality(self, correct, mistakes=0):
        """
        Calculate quality of recall (0-5 scale)
        5: Perfect recall
        4: Correct with hesitation (1 mistake allowed)
        3: Correct with difficulty
        0-2: Incorrect
        """
        if not correct:
            return 0
        if mistakes == 0:
            return 5
        elif mistakes == 1:
            return 4
        else:
            return 3

    def review_item(self, item_id, quality):
        """
        Update item based on review performance
        SM-2 Algorithm implementation
        """
        if item_id not in self.items:
            self.add_item(item_id)

        item = self.items[item_id]

        # Record history
        item['history'].append({
            'timestamp': datetime.now().isoformat(),
            'quality': quality,
            'interval': item['interval']
        })

        # Update repetitions
        if quality >= 3:
            if item['repetitions'] == 0:
                item['interval'] = 1
            elif item['repetitions'] == 1:
                item['interval'] = 6
            else:
                item['interval'] = round(item['interval'] * item['easiness'])

            item['repetitions'] += 1
        else:
            # Reset if failed
            item['repetitions'] = 0
            item['interval'] = 1

        # Update easiness factor
        item['easiness'] = max(1.3, item['easiness'] + (0.1 - (5 - quality) * (0.08 + (5 - quality) * 0.02)))

        # Calculate next review date
        item['next_review'] = datetime.now() + timedelta(days=item['interval'])

        # Update mastery level (0-100)
        # Based on: repetitions, easiness, and recent history
        recent_quality = [h['quality'] for h in item['history'][-5:]]
        avg_recent_quality = np.mean(recent_quality) if recent_quality else 0

        item['mastery_level'] = min(100, (
            item['repetitions'] * 15 +  # Max 5 reps * 15 = 75
            (item['easiness'] - 1.3) * 20 +  # Easiness contribution
            avg_recent_quality * 5  # Recent performance
        ))

        return item

    def get_due_items(self, all_items):
        """Get items that are due for review"""
        due_items = []
        now = datetime.now()

        for item_id in all_items:
            if item_id not in self.items:
                self.add_item(item_id)

            item = self.items[item_id]
            if item['next_review'] <= now:
                due_items.append({
                    'id': item_id,
                    'priority': self._calculate_priority(item),
                    'mastery': item['mastery_level']
                })

        # Sort by priority (lower mastery = higher priority)
        due_items.sort(key=lambda x: (x['priority'], random.random()))

        return [item['id'] for item in due_items]

    def _calculate_priority(self, item):
        """Calculate review priority (lower = more urgent)"""
        overdue_days = max(0, (datetime.now() - item['next_review']).days)
        priority = (
            100 - item['mastery_level'] +  # Lower mastery = higher priority
            overdue_days * 10  # Overdue items get priority boost
        )
        return priority

    def get_statistics(self):
        """Get learning statistics"""
        if not self.items:
            return {
                'total_items': 0,
                'mastered': 0,
                'learning': 0,
                'new': 0,
                'due_now': 0
            }

        total = len(self.items)
        mastered = sum(1 for item in self.items.values() if item['mastery_level'] >= 80)
        learning = sum(1 for item in self.items.values() if 20 < item['mastery_level'] < 80)
        new = sum(1 for item in self.items.values() if item['mastery_level'] <= 20)
        due_now = sum(1 for item in self.items.values() if item['next_review'] <= datetime.now())

        return {
            'total_items': total,
            'mastered': mastered,
            'learning': learning,
            'new': new,
            'due_now': due_now
        }


class GeographyLearningSystem:
    """
    Complete geography learning system with interactive map and spaced repetition
    """

    def __init__(self):
        self.learner = SpacedRepetitionLearner()
        self.load_country_data()
        self.current_question = None
        self.session_stats = {
            'correct': 0,
            'incorrect': 0,
            'perfect': 0
        }

    def load_country_data(self):
        """Load country and capital data"""
        # Using pycountry for reliable country data
        self.countries = {}

        for country in pycountry.countries:
            # Get ISO alpha-3 code for mapping
            code = country.alpha_3
            name = country.name

            # Common name mapping for better user experience
            common_names = {
                'Bolivia, Plurinational State of': 'Bolivia',
                'Venezuela, Bolivarian Republic of': 'Venezuela',
                'Iran, Islamic Republic of': 'Iran',
                'Korea, Democratic People\'s Republic of': 'North Korea',
                'Korea, Republic of': 'South Korea',
                'Moldova, Republic of': 'Moldova',
                'Tanzania, United Republic of': 'Tanzania',
                'Viet Nam': 'Vietnam',
                'Russian Federation': 'Russia',
                'Syrian Arab Republic': 'Syria',
                'Lao People\'s Democratic Republic': 'Laos',
                'Congo, The Democratic Republic of the': 'Democratic Republic of Congo',
                'Palestinian Territory, Occupied': 'Palestine',
                'Macedonia, Republic of': 'North Macedonia',
                'Brunei Darussalam': 'Brunei',
                'Côte d\'Ivoire': 'Ivory Coast',
            }

            display_name = common_names.get(name, name)

            self.countries[code] = {
                'name': display_name,
                'official_name': name,
                'code': code
            }

        # Add capitals manually for major countries (subset for demo)
        # In production, you'd use a complete capitals database
        self.capitals = {
            'USA': 'Washington, D.C.', 'CAN': 'Ottawa', 'MEX': 'Mexico City',
            'BRA': 'Brasília', 'ARG': 'Buenos Aires', 'COL': 'Bogotá',
            'GBR': 'London', 'FRA': 'Paris', 'DEU': 'Berlin', 'ITA': 'Rome',
            'ESP': 'Madrid', 'POL': 'Warsaw', 'UKR': 'Kyiv', 'ROU': 'Bucharest',
            'NLD': 'Amsterdam', 'BEL': 'Brussels', 'GRC': 'Athens', 'PRT': 'Lisbon',
            'SWE': 'Stockholm', 'NOR': 'Oslo', 'FIN': 'Helsinki', 'DNK': 'Copenhagen',
            'RUS': 'Moscow', 'CHN': 'Beijing', 'JPN': 'Tokyo', 'IND': 'New Delhi',
            'IDN': 'Jakarta', 'THA': 'Bangkok', 'VNM': 'Hanoi', 'PHL': 'Manila',
            'KOR': 'Seoul', 'PRK': 'Pyongyang', 'PAK': 'Islamabad', 'BGD': 'Dhaka',
            'IRN': 'Tehran', 'IRQ': 'Baghdad', 'SAU': 'Riyadh', 'TUR': 'Ankara',
            'EGY': 'Cairo', 'ZAF': 'Pretoria', 'NGA': 'Abuja', 'KEN': 'Nairobi',
            'ETH': 'Addis Ababa', 'TZA': 'Dodoma', 'GHA': 'Accra', 'DZA': 'Algiers',
            'MAR': 'Rabat', 'AUS': 'Canberra', 'NZL': 'Wellington', 'CHL': 'Santiago',
            'PER': 'Lima', 'VEN': 'Caracas', 'CUB': 'Havana', 'ISR': 'Jerusalem',
            'ARE': 'Abu Dhabi', 'QAT': 'Doha', 'KWT': 'Kuwait City', 'JOR': 'Amman',
            'AFG': 'Kabul', 'MMR': 'Naypyidaw', 'NPL': 'Kathmandu', 'LKA': 'Colombo',
            'MYS': 'Kuala Lumpur', 'SGP': 'Singapore', 'AUT': 'Vienna', 'CHE': 'Bern',
            'CZE': 'Prague', 'HUN': 'Budapest', 'BGR': 'Sofia', 'HRV': 'Zagreb',
            'SRB': 'Belgrade', 'SVK': 'Bratislava', 'SVN': 'Ljubljana', 'EST': 'Tallinn',
            'LVA': 'Riga', 'LTU': 'Vilnius', 'BLR': 'Minsk', 'IRL': 'Dublin',
            'ISL': 'Reykjavik', 'LUX': 'Luxembourg', 'MLT': 'Valletta', 'CYP': 'Nicosia'
        }

        # Add capitals to country data
        for code, capital in self.capitals.items():
            if code in self.countries:
                self.countries[code]['capital'] = capital

        # Filter to only countries with capitals for learning
        self.countries_with_capitals = {
            code: data for code, data in self.countries.items()
            if 'capital' in data
        }

    def create_world_map(self, highlight_country=None, color_by_mastery=False):
        """Create interactive world map with Plotly"""

        if color_by_mastery:
            # Color countries by mastery level
            colors = []
            hover_texts = []

            for code in self.countries.keys():
                if code in self.learner.items:
                    mastery = self.learner.items[code]['mastery_level']
                    colors.append(mastery)
                    hover_text = f"{self.countries[code]['name']}<br>Mastery: {mastery:.0f}%"
                    if code in self.capitals:
                        hover_text += f"<br>Capital: {self.capitals[code]}"
                    hover_texts.append(hover_text)
                else:
                    colors.append(0)
                    hover_text = f"{self.countries[code]['name']}<br>Not yet studied"
                    if code in self.capitals:
                        hover_text += f"<br>Capital: {self.capitals[code]}"
                    hover_texts.append(hover_text)

            fig = go.Figure(data=go.Choropleth(
                locations=list(self.countries.keys()),
                z=colors,
                text=hover_texts,
                hoverinfo='text',
                colorscale=[
                    [0, '#ef5350'],      # Red - Not learned
                    [0.2, '#ff7043'],    # Orange-red
                    [0.4, '#ffa726'],    # Orange
                    [0.6, '#ffee58'],    # Yellow
                    [0.8, '#9ccc65'],    # Light green
                    [1, '#66bb6a']       # Green - Mastered
                ],
                colorbar=dict(
                    title="Mastery %",
                    tickvals=[0, 25, 50, 75, 100],
                    ticktext=['0%', '25%', '50%', '75%', '100%']
                ),
                marker_line_width=0.5,
                marker_line_color='white'
            ))

        else:
            # Simple map with single country highlighted
            colors = []
            for code in self.countries.keys():
                if code == highlight_country:
                    colors.append(1)
                else:
                    colors.append(0)

            hover_texts = [
                f"{self.countries[code]['name']}" +
                (f"<br>Capital: {self.capitals[code]}" if code in self.capitals else "")
                for code in self.countries.keys()
            ]

            fig = go.Figure(data=go.Choropleth(
                locations=list(self.countries.keys()),
                z=colors,
                text=hover_texts,
                hoverinfo='text',
                colorscale=[[0, '#e0e0e0'], [1, '#2196f3']],
                showscale=False,
                marker_line_width=0.5,
                marker_line_color='white'
            ))

        fig.update_layout(
            geo=dict(
                showframe=False,
                showcoastlines=True,
                projection_type='natural earth',
                bgcolor='rgba(0,0,0,0)'
            ),
            margin=dict(l=0, r=0, t=0, b=0),
            height=500,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )

        return fig

    def get_next_question(self, question_type='country'):
        """Get next question using spaced repetition"""
        available_items = list(self.countries_with_capitals.keys())
        due_items = self.learner.get_due_items(available_items)

        if not due_items:
            # If nothing due, pick random items prioritizing unstudied
            unstudied = [item for item in available_items if item not in self.learner.items]
            if unstudied:
                selected = random.choice(unstudied)
            else:
                selected = random.choice(available_items)
        else:
            # Pick from due items (already sorted by priority)
            selected = due_items[0]

        self.current_question = {
            'country_code': selected,
            'country_name': self.countries[selected]['name'],
            'capital': self.capitals[selected],
            'type': question_type,
            'start_time': datetime.now()
        }

        return self.current_question

    def check_answer(self, user_answer, question_type='country'):
        """Check answer with fuzzy matching (max 1 mistake allowed)"""
        if not self.current_question:
            return None

        # Get correct answer
        if question_type == 'country':
            correct_answer = self.current_question['country_name']
        else:
            correct_answer = self.current_question['capital']

        # Normalize answers
        user_answer = user_answer.strip().lower()
        correct_answer_lower = correct_answer.lower()

        # Check exact match
        if user_answer == correct_answer_lower:
            mistakes = 0
            correct = True
        else:
            # Calculate Levenshtein distance
            distance = Levenshtein.distance(user_answer, correct_answer_lower)
            mistakes = distance
            correct = distance <= 1  # Allow max 1 mistake

        # Calculate quality for spaced repetition
        quality = self.learner.calculate_quality(correct, mistakes)

        # Update learning system
        item_id = self.current_question['country_code']
        self.learner.review_item(item_id, quality)

        # Update session stats
        if correct:
            self.session_stats['correct'] += 1
            if mistakes == 0:
                self.session_stats['perfect'] += 1
        else:
            self.session_stats['incorrect'] += 1

        result = {
            'correct': correct,
            'mistakes': mistakes,
            'user_answer': user_answer,
            'correct_answer': correct_answer,
            'quality': quality,
            'country_code': item_id,
            'country_name': self.current_question['country_name'],
            'capital': self.current_question['capital']
        }

        return result

    def get_progress_stats(self):
        """Get detailed progress statistics"""
        stats = self.learner.get_statistics()

        # Calculate mastery distribution
        mastery_levels = [item['mastery_level'] for item in self.learner.items.values()]

        return {
            **stats,
            'avg_mastery': np.mean(mastery_levels) if mastery_levels else 0,
            'session_correct': self.session_stats['correct'],
            'session_incorrect': self.session_stats['incorrect'],
            'session_perfect': self.session_stats['perfect'],
            'session_accuracy': (
                self.session_stats['correct'] /
                max(1, self.session_stats['correct'] + self.session_stats['incorrect'])
            ) * 100
        }


# Initialize the learning system
geo_system = GeographyLearningSystem()

# Global state for Gradio
current_question = None
question_type_state = 'country'

def start_new_question(q_type):
    """Start a new question"""
    global current_question, question_type_state
    question_type_state = q_type

    question = geo_system.get_next_question(q_type)
    current_question = question

    # Create map with highlighted country
    fig = geo_system.create_world_map(highlight_country=question['country_code'])

    if q_type == 'country':
        prompt = f"🌍 **What country is highlighted on the map?**\n\nCapital: {question['capital']}"
    else:
        prompt = f"🏛️ **What is the capital of {question['country_name']}?**"

    return fig, prompt, "", ""  # figure, question, answer_box (cleared), result

def submit_answer(user_answer):
    """Submit and check answer"""
    global current_question, question_type_state

    if not current_question:
        return None, "⚠️ Please start a new question first!", ""

    result = geo_system.check_answer(user_answer, question_type_state)

    if result['correct']:
        if result['mistakes'] == 0:
            feedback = f"✅ **Perfect!** The answer is **{result['correct_answer']}**\n\n"
            feedback += f"🎯 Quality Score: {result['quality']}/5 (Perfect recall)\n"
            feedback += f"📍 {result['country_name']} - Capital: {result['capital']}"
        else:
            feedback = f"✅ **Correct!** (with minor typo)\n\nYou wrote: {result['user_answer']}\n"
            feedback += f"Correct spelling: **{result['correct_answer']}**\n\n"
            feedback += f"🎯 Quality Score: {result['quality']}/5\n"
            feedback += f"📍 {result['country_name']} - Capital: {result['capital']}"
    else:
        feedback = f"❌ **Not quite!**\n\nYou wrote: {result['user_answer']}\n"
        feedback += f"Correct answer: **{result['correct_answer']}**\n\n"
        feedback += f"🎯 Quality Score: {result['quality']}/5 (Needs review)\n"
        feedback += f"📍 {result['country_name']} - Capital: {result['capital']}\n\n"
        feedback += "💡 This country will be shown again soon for practice!"

    # Get updated stats
    stats = geo_system.get_progress_stats()
    stats_text = f"""
    ### 📊 Session Stats
    - Correct: {stats['session_correct']} ({stats['session_perfect']} perfect)
    - Incorrect: {stats['session_incorrect']}
    - Accuracy: {stats['session_accuracy']:.1f}%

    ### 🎓 Overall Progress
    - Total Countries: {stats['total_items']}
    - Mastered (80%+): {stats['mastered']} 🌟
    - Learning: {stats['learning']} 📚
    - New: {stats['new']} 🆕
    - Due for Review: {stats['due_now']} ⏰
    - Average Mastery: {stats['avg_mastery']:.1f}%
    """

    return None, feedback, stats_text

def show_progress_map():
    """Show world map colored by mastery"""
    fig = geo_system.create_world_map(color_by_mastery=True)
    stats = geo_system.get_progress_stats()

    summary = f"""
    ## 🗺️ Your World Knowledge Map

    **Mastery Levels:**
    - 🟢 Green (80-100%): Mastered
    - 🟡 Yellow (50-80%): Learning well
    - 🟠 Orange (20-50%): Needs practice
    - 🔴 Red (0-20%): New or struggling

    **Your Stats:**
    - Countries studied: {stats['total_items']}/{len(geo_system.countries_with_capitals)}
    - Mastered: {stats['mastered']}
    - Average mastery: {stats['avg_mastery']:.1f}%
    - Due for review now: {stats['due_now']}
    """

    return fig, summary

# Create Gradio Interface
with gr.Blocks(title="🌍 World Geography Master", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🌍 World Geography Master
    ### Learn ALL countries and capitals with adaptive spaced repetition!

    **How it works:**
    - ✅ Answer correctly 3 times → Country is mastered!
    - ❌ Make mistakes → System shows it more frequently
    - 🎯 Max 1 spelling mistake allowed
    - 🧠 Smart algorithm optimizes your learning schedule
    - 📈 Track your progress on the world map!

    **Study modes:** Country name quiz or Capital quiz
    """)

    with gr.Tab("🎮 Practice"):
        with gr.Row():
            with gr.Column(scale=2):
                map_output = gr.Plot(label="World Map")

            with gr.Column(scale=1):
                question_display = gr.Markdown("Click a button below to start!")

                with gr.Row():
                    country_btn = gr.Button("🌍 Name the Country", variant="primary", size="lg")
                    capital_btn = gr.Button("🏛️ Name the Capital", variant="secondary", size="lg")

                answer_input = gr.Textbox(
                    label="Your Answer",
                    placeholder="Type the country/capital name...",
                    scale=2
                )

                submit_btn = gr.Button("✓ Submit Answer", variant="primary", size="lg")

                result_display = gr.Markdown("")

        with gr.Row():
            stats_display = gr.Markdown("""
            ### 📊 Session Stats
            Start practicing to see your stats!
            """)

    with gr.Tab("📈 Progress Dashboard"):
        with gr.Row():
            progress_btn = gr.Button("🔄 Refresh Progress Map", variant="primary")

        progress_map = gr.Plot(label="Mastery World Map")
        progress_summary = gr.Markdown("")

        progress_btn.click(
            fn=show_progress_map,
            outputs=[progress_map, progress_summary]
        )

    with gr.Tab("ℹ️ How It Works"):
        gr.Markdown("""
        ## 🧠 Spaced Repetition Algorithm (SM-2)

        This app uses the **SuperMemo 2 algorithm**, proven to optimize long-term retention:

        ### Learning Stages:

        1. **New** (0-20% mastery)
           - First time seeing this country
           - Will be shown again within 1 day

        2. **Learning** (20-80% mastery)
           - Making progress!
           - Review intervals: 1 day → 6 days → increasing
           - Mistakes reset the interval

        3. **Mastered** (80-100% mastery)
           - You know this well!
           - Review intervals extend to weeks/months

        ### Quality Scoring:
        - **5/5**: Perfect answer (no mistakes)
        - **4/5**: Correct with 1 spelling mistake
        - **3/5**: Correct but slow/hesitant
        - **0-2/5**: Incorrect (needs immediate review)

        ### Fuzzy Matching:
        - Max **1 letter mistake** allowed
        - "Wasington" → Accepted for "Washington"
        - "Brazilia" → Accepted for "Brasília"
        - "Parris" → NOT accepted (2 mistakes)

        ### Smart Scheduling:
        - **Due now**: Countries ready for review
        - **Priority boost**: Overdue items and low mastery
        - **Optimal timing**: Review just before you'd forget!

        ## 🎯 Study Tips:

        1. **Practice daily** for 10-15 minutes
        2. **Don't skip reviews** when items are due
        3. **Perfect answers** lead to faster mastery
        4. **Mistakes are OK** - they help the system adapt
        5. **Mix modes** - practice both countries and capitals

        ## 📊 Coverage:

        Currently includes **{} countries** with capitals from all continents!
        """.format(len(geo_system.countries_with_capitals)))

    # Event handlers
    country_btn.click(
        fn=lambda: start_new_question('country'),
        outputs=[map_output, question_display, answer_input, result_display]
    )

    capital_btn.click(
        fn=lambda: start_new_question('capital'),
        outputs=[map_output, question_display, answer_input, result_display]
    )

    submit_btn.click(
        fn=submit_answer,
        inputs=[answer_input],
        outputs=[map_output, result_display, stats_display]
    )

    # Allow Enter key to submit
    answer_input.submit(
        fn=submit_answer,
        inputs=[answer_input],
        outputs=[map_output, result_display, stats_display]
    )

# Launch
demo.launch(share=True, debug=True)

/tmp/ipython-input-641297544.py:526: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="🌍 World Geography Master", theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3489d71359e170e720.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
